In [4]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import pandas as pd

In [9]:
def split_path(data, mode='id'):
    if mode == 'class':
        return data.split('/')[0]
    return data.split('/')[-1]

def format_path(path):
    return f'C:/Users/kurt_/Desktop/food-101/images/{path}.jpg'

In [10]:
import pandas as pd

# Load dataset
train = pd.read_csv(r'C:\Users\kurt_\Desktop\food-101\meta\train.txt', header=None, names=['path'])
test = pd.read_csv(r'C:\Users\kurt_\Desktop\food-101\meta\test.txt', header=None, names=['path'])


In [11]:
# Process data
train['label'] = train['path'].map(lambda x: split_path(x, 'class')) 
train['idx'] = train['path'].map(split_path) 
test['label'] = test['path'].map(lambda x: split_path(x, 'class')) 
test['idx'] = test['path'].map(split_path) 

# Tüm sınıfları içeren bir listeyi oluştur
classes = train['label'].unique()

# Veri yolu formatını güncelle
train['path'] = train['path'].apply(format_path)
test['path'] = test['path'].apply(format_path)


In [14]:
import os
import shutil

def copy_files(data, destination):
    # Verilen veri kümesindeki dosyaları ilgili sınıflara göre kopyalar
    for index, row in data.iterrows():
        src = row['path']
        label = row['label']
        class_folder = os.path.join(destination, label)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        dst = os.path.join(class_folder, row['idx'] + '.jpg')
        shutil.copy(src, dst)

def create_folders(path):
    # Verilen yol altında sınıf adlarıyla klasörler oluşturur
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        print("Klasör zaten var!")

# Train ve Test için klasörler oluşturalım
train_data_path = 'C:/Users/kurt_/Desktop/food-101/train'
test_data_path = 'C:/Users/kurt_/Desktop/food-101/test'

create_folders(train_data_path)
create_folders(test_data_path)

# Dosyaları kopyalayalım
copy_files(train, train_data_path)
copy_files(test, test_data_path)

In [15]:
def count_images_in_classes(folder_path):
    class_counts = {}
    for class_name in os.listdir(folder_path):
        class_dir = os.path.join(folder_path, class_name)
        if os.path.isdir(class_dir):
            num_images = len([file for file in os.listdir(class_dir) if file.endswith('.jpg')])
            class_counts[class_name] = num_images
    return class_counts

# Train ve test klasörlerindeki sınıf adlarını ve resim sayılarını yazdıralım
train_class_counts = count_images_in_classes(train_data_path)
test_class_counts = count_images_in_classes(test_data_path)

print("Train Klasöründeki Sınıflar ve Resim Sayıları:")
for class_name, count in train_class_counts.items():
    print(f"{class_name}: {count} resim")

print("\nTest Klasöründeki Sınıflar ve Resim Sayıları:")
for class_name, count in test_class_counts.items():
    print(f"{class_name}: {count} resim")


Train Klasöründeki Sınıflar ve Resim Sayıları:
apple_pie: 750 resim
baby_back_ribs: 750 resim
baklava: 750 resim
beef_carpaccio: 750 resim
beef_tartare: 750 resim
beet_salad: 750 resim
beignets: 750 resim
bibimbap: 750 resim
bread_pudding: 750 resim
breakfast_burrito: 750 resim
bruschetta: 750 resim
caesar_salad: 750 resim
cannoli: 750 resim
caprese_salad: 750 resim
carrot_cake: 750 resim
ceviche: 750 resim
cheesecake: 750 resim
cheese_plate: 750 resim
chicken_curry: 750 resim
chicken_quesadilla: 750 resim
chicken_wings: 750 resim
chocolate_cake: 750 resim
chocolate_mousse: 750 resim
churros: 750 resim
clam_chowder: 750 resim
club_sandwich: 750 resim
crab_cakes: 750 resim
creme_brulee: 750 resim
croque_madame: 750 resim
cup_cakes: 750 resim
deviled_eggs: 750 resim
donuts: 750 resim
dumplings: 750 resim
edamame: 750 resim
eggs_benedict: 750 resim
escargots: 750 resim
falafel: 750 resim
filet_mignon: 750 resim
fish_and_chips: 750 resim
foie_gras: 750 resim
french_fries: 750 resim
french_

In [16]:
def find_duplicate_images(train_folder, test_folder):
    duplicate_found = False

    for class_name in os.listdir(train_folder):
        train_class_dir = os.path.join(train_folder, class_name)
        test_class_dir = os.path.join(test_folder, class_name)

        if os.path.isdir(train_class_dir) and os.path.isdir(test_class_dir):
            train_images = set(os.listdir(train_class_dir))
            test_images = set(os.listdir(test_class_dir))

            common_images = train_images.intersection(test_images)
            for image_file in common_images:
                train_image_path = os.path.join(train_class_dir, image_file)
                test_image_path = os.path.join(test_class_dir, image_file)
                if compare_images(train_image_path, test_image_path):
                    duplicate_found = True
                    break

    if duplicate_found:
        print("Train ve test klasörleri arasında aynı içeriğe sahip resim dosyaları bulundu.")
    else:
        print("Train ve test klasörleri arasında aynı içeriğe sahip resim dosyası bulunamadı.")

# Train ve test klasörleri arasında yinelenen resim dosyalarını kontrol edelim
find_duplicate_images(train_data_path, test_data_path)


Train ve test klasörleri arasında aynı içeriğe sahip resim dosyası bulunamadı.


In [17]:
def get_first_image_per_class(folder_path):
    first_images = []

    # Klasördeki sınıfları sıralayalım
    classes = sorted(os.listdir(folder_path))

    # Her sınıfın ilk resmini alalım
    for class_name in classes:
        class_dir = os.path.join(folder_path, class_name)
        if os.path.isdir(class_dir):
            images = os.listdir(class_dir)
            if images:  # Eğer sınıf klasöründe resim varsa
                image_path = os.path.join(class_dir, images[0])
                first_images.append(image_path)

    return first_images

# Train klasöründeki ilk sınıfların ilk resimlerini alalım
train_first_images = get_first_image_per_class(train_data_path)

# Test klasöründeki ilk sınıfların ilk resimlerini alalım
test_first_images = get_first_image_per_class(test_data_path)

# Sonuçları ekrana yazdıralım
print("Train klasöründeki ilk sınıfların ilk resimleri:")
for idx, image_path in enumerate(train_first_images, start=1):
    print(f"{idx}. resim: {image_path}")

print("\nTest klasöründeki ilk sınıfların ilk resimleri:")
for idx, image_path in enumerate(test_first_images, start=1):
    print(f"{idx}. resim: {image_path}")


Train klasöründeki ilk sınıfların ilk resimleri:
1. resim: C:/Users/kurt_/Desktop/food-101/train\apple_pie\1005649.jpg
2. resim: C:/Users/kurt_/Desktop/food-101/train\baby_back_ribs\1005293.jpg
3. resim: C:/Users/kurt_/Desktop/food-101/train\baklava\1006121.jpg
4. resim: C:/Users/kurt_/Desktop/food-101/train\beef_carpaccio\1011469.jpg
5. resim: C:/Users/kurt_/Desktop/food-101/train\beef_tartare\1000431.jpg
6. resim: C:/Users/kurt_/Desktop/food-101/train\beet_salad\1003501.jpg
7. resim: C:/Users/kurt_/Desktop/food-101/train\beignets\1000911.jpg
8. resim: C:/Users/kurt_/Desktop/food-101/train\bibimbap\1002297.jpg
9. resim: C:/Users/kurt_/Desktop/food-101/train\bread_pudding\1004499.jpg
10. resim: C:/Users/kurt_/Desktop/food-101/train\breakfast_burrito\1000351.jpg
11. resim: C:/Users/kurt_/Desktop/food-101/train\bruschetta\1003796.jpg
12. resim: C:/Users/kurt_/Desktop/food-101/train\caesar_salad\1000016.jpg
13. resim: C:/Users/kurt_/Desktop/food-101/train\cannoli\1007970.jpg
14. resim: C:

In [18]:
def count_classes(folder_path):
    num_classes = len([name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))])
    return num_classes

# Train klasöründeki sınıf sayısını alalım
num_train_classes = count_classes(train_data_path)

# Test klasöründeki sınıf sayısını alalım
num_test_classes = count_classes(test_data_path)

print(f"Train klasöründeki sınıf sayısı: {num_train_classes}")
print(f"Test klasöründeki sınıf sayısı: {num_test_classes}")


Train klasöründeki sınıf sayısı: 101
Test klasöründeki sınıf sayısı: 101


In [20]:
import os

def count_images_in_folder(folder_path):
    total_images = 0
    for root, dirs, files in os.walk(folder_path):
        total_images += len(files)
    return total_images

# Train ve test klasörlerindeki toplam resim sayısını alalım
total_train_images = count_images_in_folder(train_data_path)
total_test_images = count_images_in_folder(test_data_path)

print("Train klasöründeki toplam resim sayısı:", total_train_images)
print("Test klasöründeki toplam resim sayısı:", total_test_images)


Train klasöründeki toplam resim sayısı: 75750
Test klasöründeki toplam resim sayısı: 25250


In [24]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K

# Clear Keras session
K.clear_session()

# Define constants
n_classes = 101
batch_size = 16
width, height = 331, 331
train_data_path = r"C:\Users\kurt_\Desktop\food-101\train"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"
train_samples = 75750
test_samples = 25250

# Data generators
train_data_gen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest', rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, brightness_range=[0.6, 1.4])
test_data_gen = ImageDataGenerator(rescale=1. / 255)
train_gen = train_data_gen.flow_from_directory(train_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical')
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical')

# Function to create model
def create_model(learning_rate=0.0001, momentum=0.9, decay=1e-6, dropout_rate=0.2, l2_reg=0.005):
    # Base InceptionV3 model
    inception = InceptionV3(weights='imagenet', include_top=False)
    layer = inception.output
    layer = GlobalAveragePooling2D()(layer)
    layer = Dense(128, activation='relu')(layer)
    layer = Dropout(dropout_rate)(layer)
    predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(l2_reg), activation='softmax')(layer)
    model = Model(inputs=inception.input, outputs=predictions)

    # Compile model
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the grid search parameters
param_grid = {
    'learning_rate': [0.001, 0.0001, 0.00001],
    'momentum': [0.9, 0.95],
    'dropout_rate': [0.2, 0.3],
    'l2_reg': [0.001, 0.005, 0.01]
}

# Define a DataFrame to store epoch-wise metrics
epoch_metrics = pd.DataFrame(columns=['Model Params', 'Epoch', 'Loss', 'Accuracy', 'Val Loss', 'Val Accuracy'])

# Define variables to store best model and history
best_model = None
best_history = None
best_accuracy = 0.0

# Iterate over all parameter combinations
for lr in [0.001, 0.0001, 0.00001]:
    for momentum in [0.9, 0.95]:
        for dropout_rate in [0.2, 0.3]:
            for l2_reg in [0.001, 0.005, 0.01]:
                params = {'learning_rate': lr, 'momentum': momentum, 'dropout_rate': dropout_rate, 'l2_reg': l2_reg}
                print("Training model with parameters:", params)
                
                # Create model with current parameters
                model = create_model(**params)
                
                # Define callbacks
                checkpoint_path = f"best_model_{n_classes}class_inceptionv3den9.hdf5"
                checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
                csv_logger = CSVLogger('history_101class_inceptionv3den9.log', append=True)
                
                # Train model
                history = model.fit(train_gen,
                                    steps_per_epoch=train_samples // batch_size,
                                    validation_data=test_gen,
                                    validation_steps=test_samples // batch_size,
                                    epochs=30,
                                    callbacks=[checkpoint, csv_logger],
                                    verbose=1)
                
                # Extract metrics from history
                metrics = pd.DataFrame({
                    'Epoch': np.arange(1, 31),
                    'Loss': history.history['loss'],
                    'Accuracy': history.history['accuracy'],
                    'Val Loss': history.history['val_loss'],
                    'Val Accuracy': history.history['val_accuracy']
                })
                
                # Append to epoch_metrics DataFrame
                epoch_metrics = epoch_metrics.append(metrics, ignore_index=True)
                
                # Check if this model has the best accuracy so far
                if max(history.history['val_accuracy']) > best_accuracy:
                    best_accuracy = max(history.history['val_accuracy'])
                    best_model = model
                    best_history = history

# Write epoch-wise metrics to CSV file
epoch_metrics.to_csv('epoch_metrics2.csv', index=False)


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Training model with parameters: {'learning_rate': 0.001, 'momentum': 0.9, 'dropout_rate': 0.2, 'l2_reg': 0.001}
Epoch 1/30
4734/4734 [==============================] - ETA: 0s - loss: 2.5887 - accuracy: 0.4189
Epoch 1: val_accuracy improved from -inf to 0.62187, saving model to best_model_101class_inceptionv3den9.hdf5
4734/4734 [==============================] - 1885s 397ms/step - loss: 2.5887 - accuracy: 0.4189 - val_loss: 1.5856 - val_accuracy: 0.6219
Epoch 2/30
4734/4734 [==============================] - ETA: 0s - loss: 1.5903 - accuracy: 0.6284
Epoch 2: val_accuracy improved from 0.62187 to 0.75511, saving model to best_model_101class_inceptionv3den9.hdf5
4734/4734 [==============================] - 1492s 315ms/step - loss: 1.5903 - accuracy: 0.6284 - val_loss: 1.0254 - val_accuracy: 0.7551
Epoch 3/30
4734/4734 [==============================] - ETA: 0s - loss: 1.3348 - accuracy: 0.6885
Epoch

AttributeError: 'DataFrame' object has no attribute 'append'

In [25]:
model_path = r"C:\Users\kurt_\best_model_101class_inceptionv3den9.hdf5"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"

In [26]:
# Performans metriklerini hesaplamak için fonksiyon tanımla
def calculate_metrics(model, test_gen):
    # Modelden tahminler yap
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Gerçek sınıfları al
    true_classes = test_gen.classes
    
    # Sınıf etiketlerini al
    class_labels = list(test_gen.class_indices.keys())

    # Doğruluk, hassasiyet, geri çağırma ve F1 puanını hesapla
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    return accuracy, precision, recall, f1

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Boyutları tanımla
height, width = 331, 331

# Batch boyutunu tanımla
batch_size = 32

# Test veri üreteci oluştur
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Modeli yükle
model = load_model(model_path, compile=False)

# Performans metriklerini hesapla
accuracy, precision, recall, f1_score = calculate_metrics(model, test_gen)

# Hesaplanan metrikleri yazdır
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Found 25250 images belonging to 101 classes.
790/790 [==============================] - 92s 115ms/step
Accuracy: 0.8525544554455445
Precision: 0.8559766546429569
Recall: 0.8525544554455445
F1 Score: 0.8521356822644565
